In [1]:
import asyncio
import async_timeout
from asyncio_extras.contextmanager import async_contextmanager
from asyncio_extras.file import open_async

from pathlib import Path
import winreg
from utils import *
import psutil
import aiohttp
import shelve
from collections import namedtuple
from boltons.cacheutils import cachedproperty
from boltons.strutils import camel2under
from tqdm import tqdm_notebook as tqdm
import urllib.parse

In [2]:
class cachedclassproperty(object):
    def __init__(self, func):
        self.__doc__ = getattr(func, '__doc__')
        self.func = func

    def __get__(self, obj, cls):
        value = self.func(cls)
        setattr(cls, self.func.__name__, value)
        return value

    def __repr__(self):
        cn = self.__class__.__name__
        return '<%s func=%s>' % (cn, self.func)

In [3]:
class Game:
    """Abstract base class representing a video game."""
    @classmethod
    def get_root_dir(cls, tries=5):
        try:
            return Path(get_regkey(r'HKLM', rf'SOFTWARE\WOW6432Node\Bethesda Softworks\{cls.REG_NAME}', 'installed path'))
        except FileNotFoundError:
            if tries:
                cls.ping_launcher()
                return cls.get_root_dir(tries=(tries - 1))
            else:
                raise

    @classmethod
    def ping_launcher(cls):
        """Start the Launcher and then immediately close it."""
        cls.start_steam()
        for i in range(60):
            print('waiting for 1 second')
            time.sleep(1)
            procs = map(psutil.Process, psutil.pids())
            ob_launcher, = (proc for proc in procs if proc.name() == cls.LAUNCHER_EXE)
            ob_launcher.kill()
    
    @classmethod
    def start_steam(cls):
        os.startfile(f'steam://run/{cls.STEAM_ID}')
        
    @cachedclassproperty
    def root_dir(cls):
        return cls.get_root_dir()
    
    @cachedclassproperty
    def game_exe(cls):
        return cls.root_dir / cls.GAME_EXE
    
    @cachedclassproperty
    def launcher_exe(cls):
        return cls.root_dir / cls.LAUNCHER_EXE
    
    @cachedclassproperty
    def tesxedit_exe(cls):
        return cls.root_dir / cls.TESXEDIT_EXE

class Oblivion(Game):
    REG_NAME = 'oblivion'
    STEAM_ID = '22330'
    LAUNCHER_EXE = 'OblivionLauncher.exe'
    GAME_EXE = 'Oblivion.exe'
    TESXEDIT_EXE = 'TES4Edit.exe'
    
class Config:
    DOWNLOADS_DIR = Path(r'W:\bethmetamod-dls') #TODO
    DOWNLOADS_DB = Path(r'W:\bethmetamod-dls\downloads.shelve')
    MODS_DIR = Path(r'M:\bethmetamod\mods')
    CHUNK_SIZE = 1024
    
    @cachedclassproperty
    def downloads_db(cls):
        return shelve.open(str(cls.DOWNLOADS_DB))
        

In [10]:
class Mod:
    def __init__(self):
        downloads = []
    
    async def download(self, session):
        for download in self.downloads:
            await download.download(self, session)
    
    async def preprocess(self):
        """extracting from archives, binary patching vanilla files."""
        pass
    
    @cachedclassproperty
    def mod_name(cls):
        return camel2under(cls.__name__)    
    
    @cachedproperty
    def mod_path(self):
        return Config.MODS_DIR / self.mod_name
    
    @cachedproperty
    def dl_path(self):
        return Config.DOWNLOADS_DIR / self.mod_name
    
    def modify(self):
        """
        Yield a tuple of (source path, dest_path) where source_path is an
        absolute path and dest_path is an path relative to game.root_dir
        """
        pass
    
    async def postprocess(self):
        """Edit ini files, I/O unintensive stuff that must be performed after modify."""
        pass

In [5]:
@async_contextmanager      
async def fetch(session, url, timeout=10):
    with async_timeout.timeout(timeout):
        async with session.get(url) as response:
            yield response
            
async def chunked(response, chunk_size):
    while True:
        chunk = await response.content.read(chunk_size)
        if not chunk:
            break
        yield chunk

In [6]:
DownloadInfo = namedtuple('DownloadInfo', ('url', 'filename', 'size', 'sha256'))
class Download:
    def __init__(self, url):
        self.url = url
        self.dl_info = None
        
    async def download(self, mod, session, force=False):
        try:
            self.dl_info = Config.downloads_db[(type(self.name), self.url)]
        except KeyError:
            pass
        
        async with fetch(session, self.url) as response:          
            if self.dl_info:
                dl_filename = self.dl_info.filename
                dl_size = self.dl_info.size
            else:
                try:
                    dispo = response.headers['Content-Disposition']
                    _, dl_filename = dispo.split('filename=')
                    if dl_filename.startswith(('"',"'")):
                        dl_filename = localName[1:-1]
                except:
                    *_, dl_filename = urllib.parse.urlparse(self.url).path.split('/')
                
                dl_size = response.headers.get('Context-Size')
           
                
            dl_path = mod.dl_path / dl_filename
            if self.dl_info and dl_path.exists() and sha256_path(dl_path) == self.dl_info.sha256:
                #TODO log.info
                return

            pbar = tqdm(total=dl_size)
            dl_path.parent.mkdir(parents=True, exist_ok=True)
            async with open_async(str(dl_path), 'wb') as f:
                async for chunk in chunked(response, Config.CHUNK_SIZE):
                    await f.write(chunk)
                    pbar.update(len(chunk))
            pbar.close()
            
            if not self.dl_info:
                # TODO log.info
                Config.downloads_db[(type(self).__name__, self.url)] = self.dl_info = DownloadInfo(
                    url=self.url,
                    filename=dl_filename,
                    sha256=sha256_path(dl_path),
                    size=dl_path.stat().st_size,
                )

In [7]:
class OBSE(Mod):
    downloads = [
        Download('http://obse.silverlock.org/download/obse_0021.zip'),
        Download('http://obse.silverlock.org/download/obse_loader.zip'),
    ]

In [9]:
async def main(loop):
    async with aiohttp.ClientSession(loop=loop) as session:
        await OBSE().download(session)
        
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))